In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set(rc={'figure.figsize':(10,7)})

In [5]:
train = pd.read_csv('../data/raw/train.csv')
test = pd.read_csv('../data/raw/test.csv')

In [6]:
train['year'] = train.month.str[:4].astype('int')
train['price_per_area'] = train['resale_price']/train['floor_area_sqm']
train['lease_duration'] = train['year']-train['lease_commence_date']
train['storey'] = (train.storey_range.str[-2:].astype('int') + train.storey_range.str[:2].astype('int'))/2
train['flat_type'] = train['flat_type'].str.replace('-', ' ') # there are some flat_type encoded as 4-room and 4 room

# flat_type as ordinal variables (either this or the one after)
train.flat_type_ord = train.flat_type.map({
    '1_room':1,
    '2_room':2,
    '3_room':3,
    '4_room':4,
    '5_room':5,
    'executive':6,
    'multi_generation': 7
})

# flat_type as categorical variable (either this or the one before)
train = pd.get_dummies(train, columns=["flat_type"], drop_first=False)
train.drop(columns = 'flat_type_multi generation') # dropping this since it is the least frequent among all flat types


train.drop(columns = ['elevation', 'eco_category'], inplace=True) # confirm drop
train.drop(columns = ['month', 'storey_range', 'block', 'street_name', 'planning_area', 'lease_commence_date'], inplace=True) # to discuss

/Users/sylvesterchin/.pyenv/versions/3.6.10/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  from ipykernel import kernelapp as app


In [8]:
test['year'] = test.month.str[:4].astype('int')
test['lease_duration'] = test['year']-test['lease_commence_date']
test['storey'] = (test.storey_range.str[-2:].astype('int') + test.storey_range.str[:2].astype('int'))/2
test['flat_type'] = test['flat_type'].str.replace('-', ' ') # there are some flat_type encoded as 4-room and 4 room

# flat_type as ordinal variables (either this or the one after)
test.flat_type_ord = test.flat_type.map({
    '1_room':1,
    '2_room':2,
    '3_room':3,
    '4_room':4,
    '5_room':5,
    'executive':6,
    'multi_generation': 7
})

# flat_type as categorical variable (either this or the one before)
test = pd.get_dummies(test, columns=["flat_type"], drop_first=False)
test.drop(columns = 'flat_type_multi generation') # dropping this since it is the least frequent among all flat types


test.drop(columns = ['elevation', 'eco_category'], inplace=True) # confirm drop
test.drop(columns = ['month', 'storey_range', 'block', 'street_name', 'planning_area', 'lease_commence_date'], inplace=True) # to discuss

/Users/sylvesterchin/.pyenv/versions/3.6.10/envs/p3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


## joining with supplementary data

simple joins for
* comm
* hawker
* mall

count number of x within

https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula

1 deg lat = 110.574km
1 deg lng = 

In [10]:
# comm = pd.read_csv("../data/raw/sg-commerical-centres.csv")
# hawker = pd.read_csv("../data/raw/sg-gov-markets-hawker-centres.csv")
# mall = pd.read_csv("../data/raw/sg-shopping-malls.csv")
# pop = pd.read_csv("../data/raw/sg-population-demographics.csv") # calculate median age
# psch = pd.read_csv("../data/raw/sg-primary-schools.csv")
# ssch = pd.read_csv("../data/raw/sg-secondary-schools.csv")
stations = pd.read_csv("../data/raw/sg-train-stations.csv")

In [12]:
import numpy as np

def haversine_np(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km


## Distance metrics

Create distance matrix for all tables since haversine is not cheap to run.

In [101]:
def count_n_nearest(df, row):
    length = df.shape[0]
    lat = [row['latitude'] for i in range(length)]
    lng = [row['longitude'] for i in range(length)]
    return haversine_np(list(df['lat']), list(df['lng']), lat, lng)

In [217]:
# overall = []
# for index, row in train[['latitude', 'longitude']].iterrows():
#     overall.append(count_n_nearest(ssch, row))

In [218]:
# ssch_dist_matrix = pd.DataFrame.from_records(overall)

In [219]:
# ssch_dist_matrix.to_csv("ssch-dist-matrix.csv")

In [223]:
# (ssch_dist_matrix < 1).sum(axis=1)

0         4
1         6
2         1
3         2
4         2
         ..
431727    3
431728    1
431729    3
431730    4
431731    2
Length: 431732, dtype: int64

## Processing population data

Calculate average age

In [145]:
def getage(row):
    if '+' in row:
        return int(row.replace('+', ''))
    else:
        nums = row.split('-')
        return (int(nums[1])+int(nums[0])) / 2.0

In [146]:
# ## pop age
# pop['age'] = pop['age_group'].apply(getage)

In [162]:
# df = pop[['subzone', 'age', 'count']].groupby(['subzone', 'age']).sum()
# df = df.reset_index()

In [175]:
# df['weight'] = df['age'] * df['count']
# df2 = df[['subzone', 'count', 'weight']].groupby(['subzone']).sum().reset_index()
# df2['avg'] = df2['weight']/df2['count']

In [177]:
# df2.to_csv("subzone-ave-age.csv")

## MRT distance metrix

Difference is that mrts that existed only after the sales are masked.

In [2]:
def count_n_nearest_mrt(df, row):
    length = df.shape[0]
    lat = [row['latitude'] for i in range(length)]
    lng = [row['longitude'] for i in range(length)]
    can_be_used = np.array([row['year'] for i in range(length)]) > df['opening_year']
    can_be_used = np.where(~can_be_used, 999999, can_be_used)

    return haversine_np(list(df['lat']), list(df['lng']), lat, lng) * can_be_used

In [17]:
overall = []
for index, row in test[['year', 'latitude', 'longitude']].iterrows():
    overall.append(count_n_nearest_mrt(stations, row))

In [18]:
stations_dist_matrix = pd.DataFrame.from_records(overall)

In [19]:
stations_dist_matrix.to_csv("test-stations-dist-matrix.csv")

In [20]:
stations_dist_matrix

,0,1,2,3,4,5,6,7,8,9,...,156,157,158,159,160,161,162,163,164,165
0,17.931722,1.360114,12.626520,8.961989,10.268181,5.674218,5.290521e+06,7.768183,16.809092,23.253553,...,2.415638e+07,2.716417e+01,11.430941,1.704113e+07,13.801122,6.958338e+06,1.114539e+07,1.360496e+07,1.433917e+07,1.221124e+07
1,7.705948,25.472982,15.734551,23.369867,17.751785,24.181357,2.941964e+07,19.126386,8.829456,1.832348,...,1.764780e+06,2.998094e+06,12.871168,8.085827e+06,14.690651,1.904367e+07,1.941520e+07,1.509580e+07,1.744322e+07,2.196486e+07
2,18.026227,1.160313,13.429919,9.964497,9.890847,6.664687,4.963394e+06,7.387492,16.880772,23.490145,...,2.438626e+07,2.745565e+07,11.646473,1.752009e+07,13.545156,6.676581e+06,1.059111e+07,1.332420e+07,1.523022e+07,1.320270e+07
3,8.970536,13.655155,4.536347,10.927147,10.697282,11.514707,1.756126e+01,10.338815,8.415325,12.237913,...,1.314118e+07,1.562492e+01,4.412751,4.847725e+00,10.665482,9.627646e+00,1.292998e+01,1.086837e+01,7.345933e+06,1.038335e+07
4,14.836578,5.202543,13.464840,12.388352,5.881964,9.615628,8.472599e+06,3.412511,13.647793,20.647736,...,2.151289e+07,2.475131e+01,9.043878,1.569249e+07,9.655836,2.938010e+06,6.536009e+06,9.401460e+06,1.582120e+07,1.503130e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107929,7.247143,16.842386,6.584790,13.932998,12.257696,14.778669,2.077391e+07,12.497660,7.120189,9.208057,...,1.008840e+07,1.238254e+01,5.727502,1.555273e+00,11.143239,1.197739e+07,1.443492e+07,1.144018e+07,8.883101e+06,1.284706e+07
107930,3.690536,15.625788,12.169380,17.206188,6.571340,16.494298,1.940529e+07,8.126958,2.489010,9.748548,...,1.055505e+07,1.393679e+01,3.905213,8.006680e+06,3.979121,8.255941e+06,8.312848e+06,4.342351e+06,1.499628e+07,1.760072e+07
107931,6.098061,14.513215,13.813123,17.728789,4.208377,16.464670,1.807417e+07,6.278384,4.955259,12.242819,...,1.299159e+07,1.642701e+07,5.129866,1.068239e+07,1.241209,6.725174e+06,5.632898e+06,1.554864e+06,1.669533e+07,1.867035e+07
107932,24.089487,5.033382,18.325169,12.602314,15.433267,9.245893,1.137402e+06,13.011001,22.936489,29.555317,...,3.045360e+07,3.349628e+01,17.713404,2.335238e+01,19.257325,1.248779e+07,1.561991e+07,1.899323e+07,1.953583e+07,1.625911e+07


## Joining

In [271]:
comm = pd.read_csv("../data/processed/comm-dist-matrix.csv")
hawker = pd.read_csv("../data/processed/hawker-dist-matrix.csv")
mall = pd.read_csv("../data/processed/mall-dist-matrix.csv")
psch = pd.read_csv("../data/processed/psch-dist-matrix.csv")
ssch = pd.read_csv("../data/processed/ssch-dist-matrix.csv")


stations = pd.read_csv("../data/processed/stations-dist-matrix.csv")
pop = pd.read_csv("../data/processed/subzone-ave-age.csv") # calculate median age

In [272]:
train['nbr_ssch_1k'] = (ssch < 1).sum(axis=1)
train['nbr_psch_1k'] = (psch < 1).sum(axis=1)
train['nbr_mall_1k'] = (mall < 1).sum(axis=1)
train['nbr_hawker_1k'] = (hawker < 1).sum(axis=1)
train['nbr_comm_1k'] = (comm < 1).sum(axis=1)
train['nbr_station_1k'] = (stations < 1).sum(axis=1) # need extra count for dups

In [273]:
train=train.join(pop[['subzone', 'avg']], lsuffix='subzone', rsuffix='subzone')

In [274]:
# drop redundant ones
train.drop(columns=['subzonesubzone', 'latitude', 'longitude', 'region'], inplace=True)

In [275]:

train = pd.get_dummies(train, columns=["town"], drop_first=False)
train = pd.get_dummies(train, columns=["flat_model"], drop_first=False)

In [276]:
train

,floor_area_sqm,resale_price,year,price_per_area,lease_duration,storey,flat_type_1 room,flat_type_2 room,flat_type_3 room,flat_type_4 room,...,flat_model_multi generation,flat_model_new generation,flat_model_premium apartment,flat_model_premium apartment loft,flat_model_premium maisonette,flat_model_simplified,flat_model_standard,flat_model_terrace,flat_model_type s1,flat_model_type s2
0,118.0,209700.0,2001,1777.118644,12,2.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,110.0,402300.0,2014,3657.272727,11,11.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,112.0,351000.0,2020,3133.928571,16,2.0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,67.0,151200.0,2000,2256.716418,20,8.0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
4,73.0,318600.0,2013,4364.383562,28,8.0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431727,101.0,238500.0,2005,2361.386139,5,2.0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
431728,95.0,376200.0,2016,3960.000000,4,14.0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
431729,67.0,255600.0,2011,3814.925373,25,2.0,0,0,1,0,...,0,1,0,0,0,0,0,0,0,0
431730,123.0,508500.0,2013,4134.146341,14,17.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
